# Treinamento de Modelos - Risco de Crédito

Este notebook realiza o treinamento de modelos de classificação e avalia a performance com métricas padrão.

In [1]:
%pip install pandas
%pip install joblib
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Importar as bibliotecas necessárias
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import os

# Carregar os dados
dados = pd.read_csv('../dados/dados_credito_tratados.csv')

# Remover coluna ID
dados = dados.drop(columns=['ID'])

# Separar variáveis
X = dados.drop(columns=['inadimplente'])
y = dados['inadimplente']

# Dividir em treino/teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_treino = scaler.fit_transform(X_treino)
X_teste = scaler.transform(X_teste)

# Balanceamento de classes usando SMOTE
smote = SMOTE(random_state=42)
X_treino_balanceado, y_treino_balanceado = smote.fit_resample(X_treino, y_treino)

# Treinamento do modelo Gradient Boosting
modelo_gbm = GradientBoostingClassifier(n_estimators=100, random_state=42)
modelo_gbm.fit(X_treino_balanceado, y_treino_balanceado)

# Treinamento do modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_treino_balanceado, y_treino_balanceado)

# Treinamento do modelo Regressão Logística
modelo_lr = LogisticRegression(solver='saga', penalty='l2', max_iter=10000)
modelo_lr.fit(X_treino_balanceado, y_treino_balanceado)

# Função para avaliar o modelo com o threshold
def avaliar_modelo(modelo, X_teste, y_teste, threshold=0.3):
    y_prob = modelo.predict_proba(X_teste)[:, 1]
    y_pred = (y_prob > threshold).astype(int)
    acuracia = accuracy_score(y_teste, y_pred)
    recall = classification_report(y_teste, y_pred, output_dict=True)['1']['recall']
    auc_roc = roc_auc_score(y_teste, y_prob)
    return acuracia, recall, auc_roc, y_pred

# Avaliar o modelo Gradient Boosting com threshold 0.3
acuracia_gbm, recall_gbm, auc_roc_gbm, y_pred_gbm = avaliar_modelo(modelo_gbm, X_teste, y_teste, threshold=0.3)

# Avaliar o modelo Random Forest com threshold 0.3
acuracia_rf, recall_rf, auc_roc_rf, y_pred_rf = avaliar_modelo(modelo_rf, X_teste, y_teste, threshold=0.3)

# Avaliar o modelo Regressão Logística com threshold 0.3
acuracia_lr, recall_lr, auc_roc_lr, y_pred_lr = avaliar_modelo(modelo_lr, X_teste, y_teste, threshold=0.3)

# Exibir os resultados para todos os modelos
print(f"Resultados do Gradient Boosting:")
print(f"Acurácia: {acuracia_gbm:.4f}")
print(f"Recall (Classe 1): {recall_gbm:.4f}")
print(f"AUC-ROC: {auc_roc_gbm:.4f}")
print("--------------------------------------------------")
print(f"Resultados do Random Forest:")
print(f"Acurácia: {acuracia_rf:.4f}")
print(f"Recall (Classe 1): {recall_rf:.4f}")
print(f"AUC-ROC: {auc_roc_rf:.4f}")
print("--------------------------------------------------")
print(f"Resultados da Regressão Logística:")
print(f"Acurácia: {acuracia_lr:.4f}")
print(f"Recall (Classe 1): {recall_lr:.4f}")
print(f"AUC-ROC: {auc_roc_lr:.4f}")

# Salvar os modelos treinados
os.makedirs('../resultados/modelos', exist_ok=True)
joblib.dump(modelo_gbm, '../resultados/modelos/modelo_gbm.pkl')
joblib.dump(modelo_rf, '../resultados/modelos/modelo_rf.pkl')
joblib.dump(modelo_lr, '../resultados/modelos/modelo_lr.pkl')



Resultados do Gradient Boosting:
Acurácia: 0.6082
Recall (Classe 1): 0.7883
AUC-ROC: 0.7683
--------------------------------------------------
Resultados do Random Forest:
Acurácia: 0.6707
Recall (Classe 1): 0.6931
AUC-ROC: 0.7501
--------------------------------------------------
Resultados da Regressão Logística:
Acurácia: 0.3848
Recall (Classe 1): 0.8949
AUC-ROC: 0.7323


['../resultados/modelos/modelo_lr.pkl']